In [1]:
import pandas as pd
import numpy as np
import datetime
import importlib
import pickle

import data_utils
import model_utils
import train_utils
import evaluation
importlib.reload(data_utils)
importlib.reload(model_utils)
importlib.reload(train_utils)
importlib.reload(evaluation)

import torch
import torch.nn as nn
from torchtext.data import TabularDataset, Field, RawField, BucketIterator, Iterator

In [6]:
cpu = True

### read data and put in batch

In [7]:
train_data, val_data, test_data = data_utils.prep_all_data(cpu=cpu)

TypeError: prep_all_data() got an unexpected keyword argument 'cpu'

### Get models

In [3]:
m = model_utils.GRU()

### Training 

In [5]:
importlib.reload(model_utils)
importlib.reload(evaluation)
m = model_utils.GRU(pretrained_emb=False)
result = train_utils.train(train_data, val_data, m, cpu=False,
                          lr=1e-2, print_freq=5, max_epoch=100)

RuntimeError: CUDA error: device-side assert triggered

In [34]:
evaluation.calculate_f1(val_data, m1)

0.817117117117117

In [19]:
evaluation.calculate_f1(test_data, m1)

0.7882596882596883

### parameter searching - MLP
best model so far: 3 middle layer, 100 hidden units, dropout=0.2 at second to last layer

saved as ./data/model_checkpoints/MLP_Jan23.mdl

### Parameter Searching Bidirectional LSTM 

### Evaluation

In [1112]:
emb = nn.Embedding(10, 3)

In [1114]:
emb.weight

Parameter containing:
tensor([[-0.0974,  0.2542,  0.8995],
        [ 0.3284, -1.3262,  0.2224],
        [-0.1773, -0.5326,  0.5931],
        [-0.4633,  0.8940,  0.3648],
        [ 2.7178, -0.6496, -1.2509],
        [ 2.0111,  0.7471, -2.4884],
        [ 0.8162,  0.1524,  0.8900],
        [-0.5783, -1.2482,  1.3301],
        [-0.9398, -0.9525, -1.8860],
        [-0.1170,  0.2772,  1.4776]], requires_grad=True)

### Generate submission file for kaggle 

In [46]:
def get_submission(m, test_data):
    m.eval()
    labels = np.load("./data/labels.npy")
    final_labels = []
    for x, y, extra in test_data:
        pred = (torch.sigmoid(m((x, extra["raw_text"]))) > 0.5).int().cpu().numpy()
        batch_size, num_class = pred.shape
        for i in range(batch_size):
            pred_idx = np.arange(num_class)[(pred[i] == 1).astype('bool')] 
            if len(pred_idx) == 0:
                pred_idx = [np.argmax(pred[i]).item()]
            pred_label = [labels[j] for j in pred_idx]
            final_labels.append(
                {"ID": int(extra["ID"][i]),
                 "CORE RELATIONS": " ".join(pred_label)})
    return pd.DataFrame(final_labels)

In [55]:
print(evaluation.calculate_f1(train_data, m1))

0.9948186528497411


In [57]:
label_df = get_submission(m1, test_data).sort_values(by="ID").set_index("ID")

In [58]:
df_test = pd.read_csv("./data/test.csv", index_col="ID")[["text", "raw_label"]]
df_train = pd.read_csv("./data/train_real.csv", index_col="ID")[["text", "raw_label"]]
df_val = pd.read_csv("./data/val.csv", index_col="ID")[["text", "raw_label"]]

In [71]:
today = datetime.datetime.now().strftime("%b%d")
label_df.to_csv("./data/submissions/{}Baseline.csv".format(today))

In [70]:
torch.save(m1, "./data/model_checkpoints/Baseline_Jan27.mdl")

In [ ]:
label_df.join(df_test)